In [114]:
import pandas as pd
import numpy as np

In [115]:
fln = "surfweer_data_2021_01_17.csv"

In [116]:
# Import the dataframe

df = pd.read_csv(fln, index_col= 0, parse_dates= ['report_date', 'post_date'])
df.reset_index(inplace=True, drop  = True)

In [117]:
# Data preparation

# correct year in the report_date column - completed 
# split wetsuit_adviezen into wetsuit, hand, schoen, and cap

In [118]:
df.head()

,report_date,report_content,day_of_week,post_date,post_id,post_title,wetsuit_adviezen_str,wetsuit_adviezen,post_url
0,2021-01-12,Dinsdag 12 januari\nHet blijft de hele dag waa...,dinsdag,2021-01-12,post-33062,Surfweer dinsdag 12 januari 2021\n\n,"Wetsuit-adviezen, De zee is +6 C\nDinsdag: 6/5...","6/5/4mm wetsuit, hand 4mm, schoen 7mm, cap 3mm",http://surfweer.nl/surf/surfweer-dinsdag-12-ja...
1,2021-01-13,Woensdag 13 januari\nIneens zijn de golven twe...,woensdag,2021-01-12,post-33062,Surfweer dinsdag 12 januari 2021\n\n,"Wetsuit-adviezen, De zee is +6 C\nDinsdag: 6/5...","Nieuwe 5/4mm wetsuit, schoen 6mm en hand 4mm, ...",http://surfweer.nl/surf/surfweer-dinsdag-12-ja...
2,2021-01-14,"Donderdag 14 januari\nDe wind is weg, maar de ...",donderdag,2021-01-12,post-33062,Surfweer dinsdag 12 januari 2021\n\n,"Wetsuit-adviezen, De zee is +6 C\nDinsdag: 6/5...","wetsuit 6mm of 5/4mm cap +1mm, hand 5mm, schoe...",http://surfweer.nl/surf/surfweer-dinsdag-12-ja...
3,NaT,Vrijdag en het weekend\nWaar de swell donderda...,vrijdag,2021-01-12,post-33062,Surfweer dinsdag 12 januari 2021\n\n,"Wetsuit-adviezen, De zee is +6 C\nDinsdag: 6/5...",vrijdag,http://surfweer.nl/surf/surfweer-dinsdag-12-ja...
4,NaT,"Wetsuit-adviezen, De zee is +6 C\nDinsdag: 6/5...","wetsuit-adviezen,",2021-01-12,post-33062,Surfweer dinsdag 12 januari 2021\n\n,"Wetsuit-adviezen, De zee is +6 C\nDinsdag: 6/5...","wetsuit-adviezen,",http://surfweer.nl/surf/surfweer-dinsdag-12-ja...


In [119]:
def update_dates(post_date, report_weekday):
    """
    """
    weekdaag = {"maandag": 0,
                "dinsdag": 1,
                "woensdag": 2,
                "donderdag": 3,
                "vrijdag": 4,
                "zaterdag": 5,
                "zondag": 6}
    try:
        post_weekday = post_date.weekday()
        report_weekday = weekdaag[report_weekday]
        days_ahead = report_weekday - post_weekday
        report_date = post_date + datetime.timedelta(days_ahead)
        
        return report_date

    except:
        return np.nan

In [120]:
df.head(10).loc[0, 'post_url']

'http://surfweer.nl/surf/surfweer-dinsdag-12-januari-2021/'

In [121]:
df['report_date_corrected'] = df.apply(
    lambda x: update_dates(x['post_date'], x['day_of_week']), axis=1)

In [122]:
# split wetsuit_adviezen

In [123]:
# steps 

# 1. replace 'en' with ','

In [124]:
# df.loc[0:10]['wetsuit_adviezen'].str.replace(r'\ben\b', ',').str.split(',')

In [158]:
# def str_to_dict_old(advice):
    
#     """
#     split the advices
#     """
#     try:
        
#         # define a function to clean special characters 
#         advice = advice.lower()
#         advice = advice.replace(',', '')
#         advice = advice.replace('-', ' ')
#         advice = advice.replace('&', ' ')
#         advice = advice.replace('schoentjes', 'schoen')
#         advice_list = advice.split()
# #         print(advice_list)
        
#         keys_list = ['wetsuit', 'hand', 'schoen', 'cap', 'of'] # 

#         # extract the recommened thickness- to be revised
#         values = [value for value in advice_list if 'mm' in value]

#         # extract the keys based on [wetsuit, hand, schoen, cap]
#         diff = list((set(advice_list) - set(values))& set(keys_list))
#         keys = [item for item in advice_list if item in diff]
        
#         return dict(zip(keys, values))
    
#     except:
        
#         return advice

 use a different strategy to split the information  
 - replace connection words using ',' (met, en) 
 - split based on ',' 
 - search for numerical values in the string 
 - for each part determine whether it is for 'wetsuit', 'hand', 'schoen', 'cap'

In [126]:
import re

In [127]:
def hasNumbers(inputString):
     return any(char.isdigit() for char in inputString)

In [149]:
def str_to_dict(advice):
    try:
        advice = clean_string(advice)
        advice_list = advice.split(',')
        advice_list = [item for item in advice_list if hasNumbers(item)]

        values = []
        keys = []

        for item in advice_list:

            if bool(re.search('([A-z]+)', item)):
                key = re.search('([A-z]+)', item)[0]
            else:
                key = 'wetsuit'

            value = re.search('([0-9]+)', item)[0]

            keys.append(key)
            values.append(value)
        
        keys[0] = 'wetsuit'
        
        return dict(zip(keys, values))
    except:
        return None

In [137]:
def clean_string(advice):
    
    advice = advice.lower()
    advice = re.sub(r'met|\ben\b|&', ',', advice)
    advice = re.sub('-', ' ', advice)
    advice = re.sub('/', '', advice)
    advice = re.sub(r'mm\b', '', advice)
    
    advice = advice.replace('schoentjes', 'schoen')
    advice = advice.replace('schoen', ',schoen') # ensure properly separated
    advice = advice.replace('handschoen', 'hand')
#     advice = advice.replace('suit', 'wetsuit')
    advice = advice.replace('een', ' ')
    advice = advice.replace('of', ' ') ##
    
    return advice

cleaned_advice = clean_string(advice)

In [150]:
df_wet = df['wetsuit_adviezen'].apply(lambda x:str_to_dict(x)).apply(pd.Series).copy()

In [151]:
df_combined = df.join(df_wet)

In [154]:
df_combined['wetsuit'].isna().mean()

0.613882863340564

In [157]:
df

,wetsuit_adviezen,wetsuit
250,maandag,NaN
251,dinsdag,NaN
252,woensdag,NaN
253,wetsuit-advies,NaN
254,Een 3/2 tje kan nog net,32
255,3/2mm of 4/3mm id avond,32
256,Een 4/3mm na het koufront!,43
257,vrijdag,NaN
258,zaterdag,NaN
259,wetsuit-advies,NaN


In [133]:
advice = "Een 6/4mm of 5mm met schoen 7mm en handschoen"

advice = clean_string(advice)
advice_list = advice.split(',')
advice_list = [item for item in advice_list if hasNumbers(item)]

values = []
keys = []

for item in advice_list:

    if bool(re.search('([A-z]+)', item)):
        key = re.search('([A-z]+)', item)[0]
    else:
        key = 'wetsuit'

    value = re.search('([0-9]+)', item)[0]

    keys.append(key)
    values.append(value)
    
dict(zip(keys, values))

{'wetsuit': '64', 'schoen': '7'}

In [134]:
advice = clean_string(advice)

In [135]:
keys

['wetsuit', 'schoen']

In [136]:
advice_list

['  64   5 ', 'schoen 7 ']